本文翻译自：
http://www.auai.org/uai2012/papers/294.pdf

# 在即时战略游戏中以有限信息对策略进行推断

## 摘要

在经典的即时战略（RTS）游戏中，敌人单位只会在它们在友方单位视野中时才是可见的。对于对手部署的知识因而受限于通过侦查能够发现的东西。由于实施侦查的单位不能干其他事，且对手会试图阻止侦查，因而相关信息是有开销的。从当前的观测到的信息中推断对手当前和未来的策略是非常重要的。我们提出了一个动态贝叶斯网络模型表示在RTS游戏，星际争霸中策略与观测指标的关系，并建立一个框架对侦查到的信息进行整合。我们展示了该模型拥有通过从可观测世界推断出未观测到世界的能力。

## 1 概论

即时战略（RTS）游戏，作为战争中经济与军事层面的游戏模拟，非常复杂而具有挑战性。玩家必须管理资源，在进行长期短期规划，移动由大致100个异质单位组成的军队，同时应对对手的行动，这一切都是在关于游戏状态的非完全信息和即时游戏的约束下进行的。我们对不完全信息下的决策特别感兴趣，特别是，玩家如何从有限的信息观测中，预测它的对手的行动？

这个工作提出了*单位数量推断*问题。在开局的时候，玩家做出建造什么单位与研究什么技术的的决策。这些决策决定了整场游戏的“感觉”：是积极进攻还是倾向防守，发展经济还是发展技术等。这些玩家的计划在开局的*建造命令*中被反映出来。由于游戏早期的资源受到限制，玩家只能走一条特定的科技线路，构建一个特定构成的军团。游戏开场的主要挑战就是制定一个开场策略，可以克制对手的开场策略以获取优势。

由于玩家的信息是受限的，从而这个任务是复杂的。敌人单位只在在友方单位在附近时才可见。玩家关于其对手策略的知识就限制在了通过侦查已经获得的信息上。试图观测敌人所有行动是不切实际的，因为这么做将从其他任务中调走太多单位，而同时敌人还会在得到任何有价值信息前试图破坏侦查。相应的，玩家必进行适当程度的侦查，然后在获得的有限知识中去推断敌方的部署。一个有效的开场策略模型必须能够从这些信息中进行推断。

我们提出了一个动态贝叶斯网络模型以建模策略并将它运用到了RTS游戏星际争霸上。模型由一个表征真实游戏状态的隐变量模型与一个观测模型组成，其描述了隐状态如何生成了可观测状态。真实状态，也被我们称为*战斗空间*，可以被敌人每个单位类型的数量所描述。不同单位类型的数量的变化被一个马尔科夫过程表示，其在给定隐策略过程的历史后条件独立。该策略决定了哪些单位在特定时间中被生成出来，以期满足特定的军团构成所要求的数量。然后，观测模型决定了给定真实状态与观测“努力”后的观测的似然度。尽管隐状态空间是非常大的，由于各数量变量的可能取值太多，模型结构采用了一个有效的粒子滤波算法来进行推断，它利用了各单位计数的条件独立性。

### 1.1 即时战略游戏

我们来简要介绍RTS游戏。RTS游戏是对战争的总体模拟。玩家建立*基地*，基地包含*资源*，通过建造*城市中心*。然后玩家使用*工人*去收集资源，资源可以用掉来建造建筑与可移动单位。我们将使用广义化的术语*单位*来同时指建筑和可移动单位。*生产建筑*生成可移动单位，其中包括工人与军事单位。相对应的，建造*技术建筑*使得建造更强大的战斗单位成为可能，并且在此基础上才能建造更高级的技术建筑。游戏会在某玩家不再控制任何建筑时结束，尽管玩家通常会在那之前认输。

大多数RTS游戏包含一个叫做战争迷雾的机制，一个关于军事指挥官并非全知全能的象征。战争迷雾使得敌人单位不可见除非它们在友方单位的视野内。缺乏关于对方行动准确信息的玩家会因此暴露在被对方突然袭击的危险中。有经验的玩家会将侦查放到很高的优先级上，并且对从侦查的信息推断对手的意图非常熟练。这个侦查过程正是我们所想要建模的。

### 1.2 相关工作

对玩家开局的完全描述需要确定在所有时刻每个单位类型的数量。在我们的知识范围里，没有人尝试过在这个粒度建立关于开局的模型。已有工作简化了这个任务，通过对开局策略的描述减少细节。

一个常见的简化是假设一个小规模的开局类型，比如“速攻”，或“种田”。Weber与Mateas(2009)为星际争霸开局建立了一个标签集，基于玩家社区志愿者，并且尝试了一些监督分类器来预测这些标签。Schadd等人(2007) 为免费RTS游戏Spring设计了一个两层标签系统。

一个对于离散分类策略的替代是直接对游戏状态动态本身建模。比如可以表示开场策略为状态空间中所经过的路径。Ponsen等人.(2007)使用了关于Wargus的建筑建造的有限状态机（FSM）模型。状态表示为哪个建筑已经被建造了以及建筑类型间的转换。Hsieh与Sun(2008)在星际争霸使用了类似的方法，但使用的是随机有限状态机，其具有从游戏数据中学到的转移概率。Dereszynski等人(2011)使用了隐马尔科夫模型（HMM）。隐状态间的转换在固定的时间区间上发生，并且观测是伯努利随机变量，其指定在当前状态建造某种单位的概率。

除了Schadd等人，上面提到的这些工作假设了玩家可以访问到诸如特定单位被建造的确切时间，而这在真实游戏中是很难观察到的。另一个例外是Synaeve与Bessiere(2011)，他们使用了一个有向图模型来描述星际争霸中策略离散类，游戏状态，观测上的联合分布。他们的游戏状态是布尔向量，表示每种单位是否已经被建造了，而各个游戏状态变量则在给定策略类型时条件独立。观测被整合进模型，通过给与观测矛盾的状态赋予0的后验概率。

观测模型（也被称作探测模型）的想法可以被用在很多地方。当我们不能观测到系统的真实状态，但却能观测到被潜在的真实状态控制的过程的结果时，观测模型就有用武之地了。作为例子，在生态模型中，我们经常对某个环境的某个物种的个体数感兴趣，但是我们只能通过田野观察者的计数获得相关信息，而这些观察者甚至可能根本找不到那个物种即使它们的确存在（MacKenzie等人，2002）。观测模型描述了隐藏状态如何决定观测状态，该关系也许是像观测者投入的努力程度之类的变量的函数。在找到数据后，人们能够找到隐藏状态的MAP估计。我们的模型精神上很类似于Hutchinson等人（2011）的“存在侦测”模型。

### 1.3 我们的贡献

这个工作在对已有工作上的提升上几个方面。第一，我们在各个时刻每个单位类型的计数层次上建模了游戏状态，这使得做出游戏决策的推断所需要的细节的条件得到满足。第二，由于个体单位计数在游戏中直接可见，我们的模型的推断不要求不能获得的数据。第三，由于我们的观测模型包含了成功侦查的因子，我们可以在有限的观测中发掘更大的价值，比如从经过努力侦查却仍然侦查不到比随便侦查而侦查不到显然提供了某些信息。

## 2 模型

模型整体（图1）是一个离散时间动态贝叶斯网络（Dean与Kanazawwa,1988），组合了两个不同的部分，*状态模型*与*观测模型*，下面我们分开论述。

<img src="image/F1.png">
图1：侦查模型的两层表示。我们使用盘子表示法（plate notation）显示出各个$P,K,U,f$与$O$变量对于每个单位类型$i$（但是注意变量不是可交换的。）。存在一个直接的连接，从$U_{t-k}^i$到$U_t^i$对每个单位类型。深色的矩形表示我们可以直接观测的变量。

### 2.1 状态模型

状态模型描述了战斗空间中的真实状态的生成过程。每场游戏对应一个发始于特定初始点的离散状态路径。令$S_t$表示时间$t$的状态。时间被离散化为30秒区间，以及玩家每30秒做出一个状态转移。我们将说“第k轮（epoch）”或“$t=k$”来表示第$k$次转移时间段初。当玩家访问状态$s$，他要决定每种单位生产多少。令$P_t^i$表示在时刻$t$决定生产的单位类型$i$的数量。我们建模的生产$k$个$i$类单位的决策可分为两步。第一步，决定是否生产单位类型$i$。然后，如果第一个决策的结果是要进行生产，则再决定生产多少个。这可以以零膨胀泊松分布（zero-inflated Possion distribution）进行建模。

$$
P(P_t^i = k \mid S_t = s)=
\begin{cases}
1 - \nu_s^i & k=0 \\
\nu_s^i \cdot Pois(k-1;\lambda_s^i) & k>0
\end{cases}
$$

这里，$v_s^i$是在状态$s$生产一个或以上$i$类单位的概率。所以$1-\nu_s^i$是产生零个单位的概率。若要生产一个以上，则要生产多少个单位取$\lambda_s^i$为速率参数的泊松分布所决定。

我们将模型分为两步决策是为了专门处理某些单位是否进行生产决策的特殊性。这对于*技术建筑*很重要，在大多数情况中，每种类型的技术建筑只需要建造一个——该建筑只是作为产生更高级单位的前置。决意生产技术建筑的决策反映了策略的选择，所以捕捉这一0/1决策时有必要的。而如果只使用单个泊松分布则无法对此情况进行自然的处理，因为没有方法控制产生0单位的概率同时又不改变剩余分布的形状。类似的，建模为伯努利随机变量也不合适，因为有时还是会产生两个这种类型的建筑。

令$\mathbf{P}_t = (P_t^i)_{i=1}^N$ 表示对于所有$N$个单位类型在$t$时刻的生产数量构成的向量。在我们的模型中，各个$P_t^i$随机变量在给定当前状态$S_t$后是条件独立的。当然，本来这些单位一起生产时有一些重要的约束，比如生产一种单位消耗掉的资源就不能用来生产其他单位。原则上，这些约束可以被整合进模型通过使用足够大数量的状态。无论如何，这与运用状态转移序列捕捉捕捉玩家的整体策略的目标相冲突。所以，状态数量必须被适当选择来平衡这些目标。

当侦查时，我们无法直接观测到单位生产，而只能观测到给定时间段内存在的单位中的一部分。为了在个体单位生产决策与侦查到的观测中建立联系，我们需要建模类型$i$在$t$时存在的单位总数$U_t^i$。它等于在$t$时刻已被（累积）生产的单位减去已被消灭（累积）的单位。每个$U_t^i$取值在$\{0,1,...,U_{\text{max}}\}$，其中$U_{\text{max}}$是可以存在的最大单位数。因为生产变量$P_t^i$可在无限集上取值，所以我们令$U_t^i = U_{\text{max}}$的情况，表示有$U_{\text{max}}$或更多个$i$类单位的状态。

单位可能有两种死亡方式，大多数是因为我们的单位打掉了他们的单位，而这些死亡，$\mathbf{K}_t = (K_t^i)_{i=1}^N$ 是可以观测到的。每个$i$类单位也拥有一个微小的概率$\theta^i$ 在每个步长遭受一个观测不到的损失。作为例子，一个建造中的被侦查的建筑可能被立即取消建造掉。以初始单位数$c^i$开始，各单位类型$i$在$t$时刻的计数被递归的定义为：

$$
\begin{align}
U_0^i &= c^i \\
U_t^i &\sim Binomial(U_{t-1}^i - K_{t-1}^i, 1 - \theta^i) + P_t^i 
\end{align}
$$

当从$U_{t-1}^i = U_{\text{max}} $的情况计算 $U_i^t$的时候，由于我们不知道$U_{t-1}^i$的确切值，所以我们当做$U_{t-1}^i$就是$U_{\text{max}}$。这种非常罕见的情况当然会使我们低估$U_t^i$，但是我们可以通过设定$U_{\text{max}}$足够大来在大多数情况中规避这一问题。

令$\mathbf{U}_t = (U_t^i)_{i=1}^N$ 为表示每种单位类型在$t$时刻存在的数量。我们称$\mathbf{U}_t$为在$t$时刻的战斗空间。

### 2.2 观察模型

*观测模型*指定了U_t在特定赋值下$\mathbf{U}_t$观测的似然度。我们定义一个*观测*为我们在时间区间内每种单位的存在数量，$\mathbf{O}_t = (O_t^i)_{i=1}^N$。我们假定我们能区分在一轮内的不同的敌方单位，但跨轮的不行。这就是说，如果我们在一轮内对观察到同一个类型的单位两次，我们知道它是同一个单位观察到两次还是两个不同的单位。于是我们可以建模这些观测为从$\mathbf{U}_t$中不放回抽样，在一轮内。但是在不同轮我们就不知道是不是同一个单位了。

如果我们观测到一个敌方单位，我们知道这个单位存在。而如果我们没观测到，有两个可能，一是这个单位本来就不存在，或者它存在但是我们没有付出足够的努力去观察到它。所以我们的模型需要整合某种侦查“努力程度”的测度。如果我们没怎么试图侦查，对单位失败的侦查对于它是否存在没有告诉我们多少信息。而如果我们花了很多精力还是没有侦察到，则它很可能本来就不存在。

我们可以利用领域知识来构造侦查努力程度的测量。在星际争霸中，玩家在主基地或“自然扩张”（natural expansion）处建造绝大多数建筑物。主基地是地图上玩家初始单位登场的地方。而自然扩张则是最“自然”的建造第二个基地的地方。因为建筑必须被保护，所以把它们建在一起可以取得战术优势。在早期游戏中，主基地与自然扩张变成了侦查最重要的区域。因为建筑很可能在那些地方。一个自然地关于侦查努力程度的测度就是这两个区域已经被看见的比例。我们将这个比例表示为$E_t$。

我们现在必须决定我们的侦查努力程度如何影响我们观测到的单位数量。我们考虑的第一个方法是将每个可能被观测到的单位当成一个独立的伯努利实验，拥有成功率$E_t$。于是观测可以看成二项随机变量构成的向量，$O_t^i \sim Binomial(U_t^i,E_t)$。

二项模型假设单位位置均匀且独立地分布于空间中。然而这个假设显然不对，因为单位倾向于聚在一起。作为例子，“工人”的首要任务是收集资源，它们通过在城市中心和资源间穿梭做到这一点。故而，几乎所有工人单位都会在城市和资源之间被找到。如果我们看见一个工人，这很可能是因为我们已经看到这个繁忙区域的一部分，我们将期待看到更多其余的工人。这造成了比二项模型可以预测的更大的方差，这个数据是*更分散*的，相比于二项分布。

我们通过设定二项分布的成功率参数服从Beta分布来应对更为分散的问题，这建立了一个Beta-二项模型(Haseman与Kupper,1979)

$$
O_t^i \sim BetaBinomial(U_t^i,\mu_t^i,\rho^i)
=
\begin{pmatrix}
U_t^i \\ O_t^i
\end{pmatrix}
\frac{B(O_t^i + \alpha_t^i, U_t^i - O_t^i + \beta_t^i)}{B(\alpha_t^i,\beta_t^i)}
$$

其中$B(x,y)$是beta函数，且

$$
\alpha_t^i = \mu_t^i \frac{1-\rho^i}{\rho^i}, \quad \beta_t^i = (1-\mu_t^i)\frac{1-\rho^i}{\rho^i}
$$

我们采用$(\mu,\rho)$作为Beta分布的参数，其中$\mu \in [0,1]$是Beta的均值而$\rho \in [0,1]$是散度参数，其可以被当做一种各单位间成功率的相关性。Beta分布是二项分布的共轭先验分布。当$\rho \to 0$，Beta-二项模型逼近于二项分布，而当$\rho$增加，密度将倾向分散到两边并最终变成双峰分布（bimodal）。该双层模型（bi-modality）捕捉了单位的“聚集性”，在是否我们观测到一个单位的条件下，剩余的在其附近的单位的观测成功率为一起很高或一起很低，但不会是中间水平。

对于每个单位类型$i$，我们学习一个映射$f^i(E_t)$，其从观测努力程度映射到$i$类的Beta-二项分布的均值与散度参数。

$$
\begin{align}
logit(\hat{\mu}_t^i) &= a_0^i + a_1^i E_t \\
logit(\hat{\rho}^i)  &= b^i.
\end{align}
$$

给定$U_t^i$下的观测$O_t^i$的似然度可以以此计算了。我们对每个不同的单位类型学习不同的映射，以使得在散度和观测的容易程度的区分成为可能。回归系数被假设在时间上是恒定的，但是$\hat{\mu}_t^i$会随着时间变换因为它依赖于$E_t$。

### 2.3 训练

所有模型变量，除了$\mathbf{S}$都是可被观测到的在训练时。因为所有其他变量都条件独立于$\mathbf{S}$给定$\mathbf{P}$，而$\mathbf{P}$是可被观测到的，所以我们可以分解整个模型为由$\mathbf{S}$和$\mathbf{P}$组成的潜在变量模型，以及一个完全的观测模型，其中包含所有剩余的参数。我们可以从简化的训练中分别学习S和P的参数。

生产过程是一个隐马尔科夫模型（HMM）（Rabiner,1990），其中$\mathbf{S}_t$是潜在状态而$P_t^1,\dots,P_t^N$是emissions。HMM的参数包含初始状态概率$P(S_0) = Multinomial(\eta_1,\dots,\eta_M)$,转移概率$P(S_t|S_{t-1} = s) = Multinomial(\pi_1^s,\dots,\pi_M^s)$以及膨胀泊松分布的参数$P(P_t^i = k >0|S_t = s) = \nu_s^i \cdot Pois(k-1;\lambda_s^i)$。我们用$\Phi$表示HMM的参数集，$\Phi = (\eta_1,\dots,\eta_M,\pi_1^1,\dots,\pi_M^M,\lambda_1^1,\dots,\lambda_M^M,\nu_1^1,\dots,\nu_M^N)$。

在训练期，我们观测$\mathbf{P}_t$。我们可以用平凡的用期望最大化算法（EM）估计$\mathbf{\Phi}$。我们初始化EM算法像下面这样：$\eta$与$\pi$参数被设为$1/M$。而$\nu$的值从$Uniform(0,1)$中抽出，而$\lambda$的值则从$Uniform(0,10)$中抽出。

未观测到的损失概率$\theta^i$被估计为类型$i$的未观测的单位损失除以单位-轮（unit-epoch）的数量（类似于“人年”）在一个这样类型的单位存在的期间。对于其存在多于100个单位-轮的单位类型，$\theta^i$以加性平滑$\hat{\theta}^i = \frac{d^i+1}{D^i+2}$，其中$d^i$是类型$i$未观测到的损失个数，而$D^i$是类型$i$的单位轮。这个平滑操作保证所有$i$单位类型拥有非零$\theta_i$甚至当数据并没有发生未观测的损失时也成立。对于那些没有出现100个单位轮以上的单位类型，我们使用了中位数进行估计。

函数给出分布$O_t^i$的函数$f^i$通过aod这个R的包进行极大似然的逻辑回归学习得到。我们拟合$\hat{\mu}^i$参数对于每个单位类型$i$，仅当该类型单位在数据集中被至少100次游戏中被观测到。我们拟合$\hat{\rho}^i$参数仅当单位类型满足$\hat{\mu}^i$的条件且至少两个该类型单位存在（不需要被观测到）于100次对战中。提出对$\hat{\rho}$这个条件的原因是如果它很少以一个以上的情况存在，则数据中的散度太小了，$\hat{\rho}^i$就会很接近0。在这个情况中，$\hat{\rho}^i$将仅仅在建模不去建造一个以上这个单位的倾向，而这是生产模型的$P_t$该做的事情。对于没有足够数据的$\hat{\mu}$与$\hat{\rho}$，使用中位数回归来估计系数。

### 2.4 推断

（这段有的算法我不了解，不确定是不是这样翻译，我主要感兴趣的是后面那段）

我们表示潜在变量在$t$时刻的切片为$X_t = \{ S_t,P_t^1,\dots,P_t^N,U_t^1,\dots,U_t^N \}$，而被观测的变量则表示为$Y_t = \{ E_t,K_t^1,\dots,K_t^N,O_t^1,\dots,O_t^N \}$。我们使用小写字母$y_t$与$x_t$表示这些变量的实现值（如$y_t$表示$t$时刻的证据）。因为每个$U_t^i$被$S_t$条件化，而$S_t$和$U_t^i$都是马尔科夫的，精确的滤波将要求表示前传信息（forward message）$\alpha_t = P(S_t,U_t^1,\dots,U_t^N)$。这是不可操作甚至对于不大数量的类型数，因为联合分布是$MU_{\text{max}}^N$。无论如何，一个关键的事实是，给定策略状态的历史，$S_{0:t} = (S_0,\dots,S_t)$，直到$t$时的模型可以分解为$N$个独立的HMM，每个追踪一个特定的单位类型。利用这个结构，我们使用Rao-Blackwellized粒子滤波（RBPF）来进行近似推断（Doucet等人2000;Murphy，2000）。

在我们的RBPF的应用中，我们抽取了$S_{0:t}$的粒子，并通过标准的HMM滤波（估计最后一个隐状态），精确地计算$P(U_t^i \mid s_{0:t})$。使用重要性抽样（importance sampling）框架，粒子在每个时间步被一个建议分布（proposal distribution）$Q(S_t)$中抽取。我们对我们的建议分布使用状态转移模型$Q(S_t) = P(S_t|s_{t-1})$。尽管这个选择无视了最近的$t$时刻的证据，但是他在抽样计算上很有效率，并且大多数截断本来也是没证据的，如果一定要说的话。

在$t=0$时，我们抽取$R$个粒子从初始状态$s_0^1,\dots,s_0^R \sim P(S_0)$。每个粒子有一个重要性权重$w_t^r = \phi (s_t^r) / Q(s_t^r)$，其中$\phi(s_t^r)$是粒子$s_t^r$的概率，给定完全模型（直到标准化）。在$t=0$时，$w_0^r = (P(y_0|s_r)P(s_r))/Q(s_r)$。对于$t>0$，每个粒子产生它的下一个状态值$s_t^i \sim Q(S_t) = P(S_t \mid s_{t-1}^i)$，然后我们更新它的权重，使用比率：

$$
w_t^{r'} = 
\frac{P(Y_t = y_t \mid y_{0:t-1,s_{0:t}^r})P(S_t = s_t^r \mid S_{t-1}=s_{t-1}^r)}
{P(S_t=s_t^r \mid S_{t-1}=s_{t-1}^r)}
$$

粒子$r$的新权重为$w_t^r = w_{t-1}^r w_t^{r'}$。

由于我们的建议分布是与$P(S_t \mid S_{t-1})$等价的（消去分母），我们只是对分子中的似然项感兴趣，$P(Y_t=y_t \mid y_{0:t-1},s_{0:t}^r)$，其可以分解为:

$$
\prod_{i=1}^N 
\left[
P(U_t^i \mid U_{t-1}^i,P_t^i,K_{t-1}^i)P(U_{t-1}^{i,r})
\cdot P(P_t^i \mid S_t=s_t^r)P(O_t^i \mid U_t^i,E_t^i)
\right].
$$

$P(U_{t-1}^{i,r})$是前传信息，其捕捉了$i$类单位的计数在$t-1$时的后验边际分布。在我们加权一个样本后，我们计算$P(U_t^{i,r}|s_{0:t}^r,y_t)$来更新它关于关于单位计数的信念并且转到切片$t+1$时。每个粒子计算$N$个这样的信息，对于每个单位类型。

粒子滤波经常包含重抽样步，其中抽出一个新的粒子集合，其成比率于当前集的权重与均匀权重。我们没有包含重抽样步骤因为我们认为这样将产生很多没有观测的截断，特别是在开始的轮中。重抽样也将缩减粒子总体的多样性，伴随着结果，如果之后的观测到来显示*先验*不像策略，则也许没有留存的粒子能够表示它们表示的很好。

## 3 实验

我们评估我们的模型，基于它推断隐藏单位计数的能力，在只给定真实游戏中玩家能获得的那部分信息下。我们使用两个不同的度量标准，对于经常出现1个以上的单位类型，比如军事单位，我们测量模型推断出正确单位数的能力。对于那些不建造或只建造一次的单位类型，如技术建筑，我们度量准确性于模型确定是否单位存在的能力。我们也特别地测试了模型推断缺席单位单位的能力，一个需要观测和状态模型合作的任务。

我们的实验使用RTS游戏星际争霸的回访数据，从“Gosu Gamers”网站，我们一共收集了509个神族对人族的回放。星际争霸有三个可以玩的种族，神族，人族与虫族，每个有不同的单位和能力。在这个工作中我们只考虑一个组合。在我们的试验中，我们采用人族玩家的视角，并试图预测人族玩家在做什么。

单位计数和观测的数据是用BWAPI库（BWAPI，2012）从回放数据中被提取出来的。我们使用BWTA区域分析库（Perkins,2010）来划分地图为几个区域，并手动区分对应于神族玩家主基地与自然扩展所在区域。

数据集被分为5部分来进行交叉检验。为了选定M（策略状态数）于生产模型中，我们比较了平均似然$P(P \mid \Phi)$在数据中对于$M=20,25,30,35$以及$40$.似然度对于$M=25,30,35$是统计等价的，于是我们选定了$M=30$，建立在我们对所学到的参数的考察与我们对星际争霸的知识。在选定$M$后，我们估计观测模型参数在所有5部分，使用它们对应的训练集。

### 3.1 基线方法

我们实现的经典的基线方法，包含两个集上的平均：每个类型的单位在每轮末于所有游戏上平均。2.仅限于包含那个单位出现过一次的游戏。在时间t，如果类型i的单位至今没被发现过，则基线预测类型$i$于$t$的平均所有游戏上的平均。如果至少被侦察到一次，则基线预测平均于单位类型出现过的游戏。基线应该表现的不那么差，因为很多开局都是相似的饿而且单位计数通常变化不大。


### 3.2 典型侦查行为

因为我们的模型使用从侦查获得观测作为它的证据，那么知道典型的侦查行为是什么样子就很重要了。存在侦查努力程度存在显著的峰，于$t=5$到$t=8$时，之后努力程度就降低到稳定的“背景”水平了（图2）。第一个显著的侦查开始于$t=4$.基于上面所示的侦查模式，我们将期望我们的模型应在$t=5,6,7,8$这些峰上工作的最好。

<img src="image/F2.png">

图2：各轮的平均侦查努力程度，与非零程度所占的比例。


### 3.3 模型分析

观测模型中logistic变换过的回归系数展示了不同类型的单位中不同的分散度。对移动单位而言，典型的$\hat{\rho}^i$值为0.3，其反应了移动单位经常成群移动的事实。而建筑的取值则更多样，对于Gateway(用来生产军事单位)，$\hat{\rho}^i$大约为0.71，展示了Gateway经常十分靠近彼此。而在另一个极端，Nexus（城市中心，储存工人开采的资源）的$\hat{\rho}^i$的值为$10^{-8}$，因为Nexi从来不会在旁边又建立一个相同的。侦查努力程度的回归系数几乎接近$1.0$，揭示了我们的侦查努力度量是侦查成功的良好解释变量。两个努力程度系数明显小于$1$则对应着该单位具有*隐形*能力，其可以被看做只能被特定的单位所观察到。对于这些单位中的一个，Observer，模型学到了更高的侦查努力降低了侦查到的概率，这看起来并不正确，除了Observer本来就难以侦察到外，这和其通常在侦查高峰度过后才上场有关，所以侦查努力程度与Observer存在是负相关关系。


### 3.4 推断单位数量

对于那些会被建造很多的常见单位，我们将希望能够推断其正确的数量。在星际争霸中，早期游戏中最常见的单位是Probe（神族工人），Dragoon与Zealot（基础军事单位）以及Gateway(用来生产Dragoons与Zealot的建筑)。图3比较了我们的模型的表现与基线模型在此任务上的表现。虚线对应着基线方法，实线对应着我们的模型的准确度，短划线对应着没有观测时的准确度。其中，对于真实计数u_t^i以及模型边际分布U_t^i，误差以以下方式给定：

$$
\epsilon_t^i = E[|U_t^i - u_t^i|]/(u_t^i + 1)
$$

<img src="image/F3.png">

图3：各种普通单位的预测的相对期望绝对误差，与基线模型的比较。误差柱是95%置信区间。

我们在Dragoons与Zealots上的预测基本好于基线，而Probe，我们的模型在最早的大多数情况下劣于基线方法，我们可以将基线方法在此的优良性质归因于Probe计数的低方差性，玩家在最开始几乎总是尽可能快的建造Probe增加资源产出，直到“饱和”点，其取决于玩家已经建立的基地数。而基线方法误差在最后的急剧增加乃至超过了我们的模型的误差显示了这是一个这样的时间点，有的玩家已经达到了饱和点，而有的还没有，而我们的模型并不因此显著的增加误差因为它包含这一信息。对于Gateway，基线方法在$t=10$到$t=12$值得注意的更好的好转可以Gateway的典型建造时间进行解释：基线方法误差第一次取得极小的$t=4$是几乎所有人开始建造它们的Gateway的时间，而精确时间和数量取决于它们的策略。基线误差的第二个取得极小的时间$t=11$无第二期Gateway建造期结束的时间。我们的模型对捕捉这个结构有些困难，因为它是平稳的。

我们的模型在没有观测值的情况下的误差曲线行为非常类似于基线，尽管基线通常有着更好的精度。我们的模型虽然也追踪平均行为，但是在策略状态的马尔科夫假设下容易对生成做出过多的估计。作为例子，我们的模型会在Dragoon还不能生产的时候就分配出可观的生产Dragoon的概率。

### 3.5 推断技术建筑

对于生产很多的单位来说，估计它们的数量是重要的，而对于技术建筑我们主要关注它们是否存在。故而，我们将我们的模型与一个适当修改过的基线模型，以0/1期望误差来在四个技术建筑中进行比较。

可以在图4中看出两种误差曲线模式。对于Robotics Facility与Obseratory，基线错误先到达极大值然后下降，而Citadel of Adun("Citadel")与Robotics Support Bay("Support Bay")，在整个开局阶段都在持续增加。我们的模型在没有观测值的时候与基线错误曲线十分接近，直到基线错误开始下降。

<img src="image/F4.png">

图4：技术建筑的预测的期望0/1误差，与基线相比较。误差柱是95%置信区间。为了清晰起见没有观测的误差柱没有画出来。

似乎可以看到，又一次，在侦查开始后，我们的模型超越了基线模型，尽管基线模型在在开局阶段的最后表现似乎又变好了。正如在3.4节的Gateway的结果，Robotics Facility在开局最后的错误率下降被解释为典型策略。大多数开局都有建造单独一个Robotics Facility在epoch 13之前。与Gateway结果对比，我们的模型可以捕捉Robotics Facility的这个时间结构，也许是因为Robotics Facility只会被建立一次，对比起Gateway一段时间内Gateway在一段时间内可能会建造多个。在另一方面，在一些开局中Support Bay经只备建造一次，在另一些中则根本不会被建造。我们将期待误差变得稳定在Support Bay建造期后，并期待看到一些对我们的模型有用的证据。值得注意的是侦察到Robotics Facility存在的概率是0.37，而侦查到Support Bay的概率仅有0.18。

### 3.6 推断单位的不存在

因为我们建模成功观测概率为侦查努力程度的函数，我们可以从*负观测*中获取信息爱推断单位*不*存在。在图5中，我们展示了我们的模型在给定不同的*horizon*时，最终轮的预测目标单位不存在的误差。作为例子，horizon取6指在$t=13$做出的观测，根据$t=6$的证据。正确的计数等于$0$，所以所有的误差都来自于过大的预测。我们看到随着负观测产生，模型降低了它的预测值。观测模型使用负观测与状态模型给定的建造时间推断出剩下来的时间不足以造出新单位。这个效应对于那些容易被侦查的单为是很强的。作为例子，Dragoons除11场对战外的所有对战中出现，而它们很容易被观测到因为它们会试图攻击侦察者。类似的，Robotics Facilities也是常见且更容易被侦察到比较起Observatories。基线模型（论文中没有展示）在这个任务中拥有更宽大的误差。

<img src="image/F5.png">

图5：给定不同horizon是，于$t=13$的预测误差。该误差仅在没有该单位存在的对局中计算。误差柱是95%置信区间。


### 3.7 案例研究

为了展示我们的模型如何被用来游戏中，我们通过一个特别的对战的细节考察它。在这个对战中，神族玩家使用了Reaver drop战术，其设计使用一个运输机运载一个被称为Reaver的强大单位到敌方后方攻击它们的工人。这个攻击可能是极有破坏性的，但也很容易被中断除非达成了突然袭击的效果。我们必须指出我们选择这个对战的原因是此局人族玩家的侦查特别有效，给了我们的模型做出有趣推断的机会。

这个策略的关键单位有Robotics Support Bay，它是建造Reaver的前置技术建筑；Shuttle，它是运载Reaver的运输载具当成还有Reaver它本身。我们可以比较Reaver drop开局与“标准”开局，它涉及建造一个Observatory。

这个对战有两个重要的侦查时点。人族玩家于$t=6$侦查了神族，离开了一回后于$t=10$又进行了一次侦查。图6显示了对于每个关键单位类型，在未来模型关于至少存在一个这个的单位的信念，在给定$t=6$与$t=10$的证据的情况下。

我们首先检查模型给定$t=6$时的证据时做出的预测，人族玩家这时侦察到了Robotics Facility，而Support Bay与Reaver还没有建好。于是对Observatory与Support Bay存在的信念$t=8$增加了。因为Observatory是“标准的”策略所会继续建造的，所以它受到了更多的信度。关于Shuttle存在的信念与Support Bay存在的信念变化趋势一直，因为Support Bay与Reaver drop战术紧密相关，而该战术又使得一个Shuttle的存在变得很可能。关于Reaver存在的信念增加的信念增加的比Support Bay的慢，因为Reaver必须在Support Bay建造完了之后才能建造。在这个对战中，Support Bay与Shuttle实际在$t=9$中被建好，而Reaver在$t=11$时被建好。

<img src="image/F6.png">

图6：四种不同的单位在未来存在的信念，分别基于$t=6$与$t=10$的证据进行的推断。其中Shuttle与Support Bay建于$t=9$，Reaver建于$t=11$，期内Observatory没有被建造。误差柱是95%置信区间。

我们现在看看模型在获取更多证据后如何修正它的预测。在$t=10$，人族玩家侦察到了Support Bay。这个毫无疑问地显示了Reaver将被建造。正如我们所期待的模型做得，关于Reaver将被生成的信念客观的增加了对比起使用6时刻的证据时所做出的预测。关于Shuttle存在的信念也增加了。

## 4 讨论

我们的模型，比较其基线模型，在数量预测与0/1预测上都表现得很好。而我们的模型是平稳的，基线模型是不平稳的可以解释多数基线模型与我们的模型表现相近或更优时的情况。与基线模型可以利用不同时间段生成模式的信息不同，我们的模型的隐状态的马尔科夫性则忽视了这些关于时间流动的信息。在早期游戏中，我们的模型预测生产的发生太早了，因为在模型眼里开头的几轮都差不多，还有一个很高的自返概率。在之后的游戏中，对时间的无知意味着它对捕捉时间依赖的在生产“暂停”存在困难。在另一方面，我们的模型原则上可以扩展到正常游戏，而基线方法则在开局之外就显得太粗糙了。

我们模型的领一个弱点是它没有整合关于单位数量的显式先验知识。作为例子，几乎永远也不会有两个Obseratory，但是我们的模型只能把此情况表示成Observatory生成状态只进入一次。通过让于$t$时生产单位数$(P_t^i)$依赖于该类单位在t-1数的存在数量$U_{t-1}^i$可以改进精度，然而也增加了潜在变量的复杂性。情况将变得更糟，如果我们想要整合前置关系，因为这将造成对于不同单位类型变得耦合，而这就破坏了我们为做出有效率的推断所以要凭依的条件独立性。

## 5 未来工作

一个显然的延续是开发一个可以在整场游戏中工作良好的模型。可能策略空间将增长的更大么人对手的行为也会开始影响我们的决策。平凡地通过增加状态扩展当前模型看上去是行不通的。一个可能的方向是尝试建构策略的层次结构来所见策略空间——在高层制订宏观战略，然后在底层通过试图实现它们。一个致力于完整游戏的模型也会需要整合资源流模型来使得关于建造速度的推理更有效率。为应对我们的行为的对方的策略的改变也是需要考虑的。

我们也对运用对手模型来优化侦查策略感兴趣。正如我们在图2所看到的，人类玩家基本都在相似的时间上取得侦查成功概率与期望信息增益的最优权衡点。人类或电脑玩家可以使用我们的模型确定什么时候那些重要的信息可以被侦察到。建模侦查的成功的概率与信息的存在性，是一个具有挑战性的任务。

## 致谢

这个工作之所以能够完成，依赖于Mark Udarbe与Thao-Trang Hoang在标记与整合的数据清洗工作中的努力。

这个工作部分受资助于ARO grant W911NF08-1-0242.本文的观点与结论来自作者，不能表示ARO或美国政府的官方观点。Jesse Hosteler也被来自ARCS foundation of Portland,OR 的奖学金所支持。
